<a href="https://colab.research.google.com/github/Jadeilson08/tcc-data-science/blob/master/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [x] Obtenha os dados de preço histórico de Bitcoin e Ethereum em um formato de tabela (por exemplo, CSV).
- [ ] Obtenha os dados do valor do dolar por dia
- [ ] Utilize a biblioteca Pandas para carregar e manipular os dados.
- [ ] Utilize a biblioteca Matplotlib para plotar os dados e visualizar as tendências.
- [ ] Utilize a biblioteca de aprendizado de máquina scikit-learn para dividir os dados em conjuntos de treinamento e teste.
- [ ] Escolha um modelo de aprendizado de máquina apropriado para séries temporais (por exemplo, ARIMA, SARIMA, Prophet) e treine-o com os dados de treinamento.
- [ ] Faça previsões com o modelo treinado utilizando os dados de teste e avaliar a precisão do modelo.
- [ ] Utilize o modelo treinado para fazer previsões futuras.

In [1]:
# importando o data_reader
!pip install -q yfinance
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin


yfin.pdr_override()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 12.5 MB/s eta 0:00:00


In [2]:
# criptomoedas analisadas: Bitcoin(BTC-USD), Ethereum(ETH-USD), Tether(USDT-USD), BNB(BNB)

#data inicial e final
start_date = '2017-08-01'
end_date = '2021-08-01'

# coletando os dataframe
df_bitcoin = pdr.get_data_yahoo('BTC-USD', start=start_date, end=end_date)
df_ethereum = pdr.get_data_yahoo('ETH-USD', start=start_date, end=end_date)
df_tether = pdr.get_data_yahoo('USDT-USD', start=start_date, end=end_date)
df_bnb = pdr.get_data_yahoo('BNB', start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
# o index de todos os dataframe ja sao as datas, sem necessidade de atualizar o indice
df_bitcoin.info()
#df_ethereum.info()
#df_tether.info()
#df_bnb.info()
''' etapa 1 realizada '''

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1461 entries, 2017-08-01 00:00:00+00:00 to 2021-07-31 00:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1461 non-null   float64
 1   High       1461 non-null   float64
 2   Low        1461 non-null   float64
 3   Close      1461 non-null   float64
 4   Adj Close  1461 non-null   float64
 5   Volume     1461 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 112.2 KB


' etapa 1 realizada '